# Laboratorio 2 parcial 1

## Desarrollo de una herramienta analítica usando paquetes especializados para análisis de datos en Python

Para el desarrollo de esta actividad puedes utilizar cualquier librería externa. Te recomendamos leer por completo el enunciado del laboratorio antes de comenzar, de forma que tengas claro el propósito global de la actividad y puedas desarrollar tu solución apuntando a él desde el inicio.

Al desarrollar este laboratorio pondrás a prueba tus habilidades para:

1. Identificar y abordar preguntas de negocio y de *analytics*.
2. Leer datos desde archivos y almacenarlos utilizando métodos de librerías especializadas.
3. Explorar, modificar, limpiar y unir objetos tablas de datos.
4. Implementar análisis combinando métricas descriptivas, visualización, filtrado y agrupación.
5. Implementar análisis basado en modelos estadísticos o de *machine learning*.

##  Contexto: desigualdad y factores de éxito en pruebas Saber 11 en Colombia

El ICFES es el Instituto Colombiano para el Fomento de la Educación Superior y está adscrito al Ministerio de Educación a nivel nacional. Como parte de sus funciones, el ICFES administra las pruebas Saber 11, las cuales evalúan a todos los estudiantes del país al final de su educación secundaria. El examen contiene preguntas que evalúan una variedad de áreas del conocimiento (ej., matemáticas, física, inglés, etc.) y se lleva a cabo dos veces al año, ajustándose a los diferentes calendarios académicos que siguen las instituciones educativas. Al momento de inscribirse a las pruebas, los estudiantes diligencian un formulario que recoge información sociodemográfica y relacionada con la institución a la que pertenecen. El fin es obtener información con respecto al desempeño de los estudiantes en la prueba y de sus características.

Al igual que otros países de la región, Colombia tiene grandes retos en términos de desigualdad, particularmente en el contexto de educación primaria y secundaria. Por esta razón, para el Estado colombiano es muy valioso el amplio registro de datos que el ICFES genera alrededor de las pruebas Saber 11, pues con ellos se pueden generar análisis sobre la calidad de la educación en el país y eventualmente dar lugar a recomendaciones sobre políticas públicas. En particular, la problemática a abordar en este caso de estudio es la desigualdad y factores de éxito en las pruebas Saber 11. 

Los objetivos de este caso de estudio son:

* Entender el contenido de los archivos de datos proporcionados sobre las pruebas Saber 11, generar un reporte acerca de sus características principales y seleccionar las partes de dicho contenido que podrían ser relevantes para el análisis.


* Identificar características de las variables de interés y relaciones entre ellas, por ejemplo, a través de agrupación, visualizaciones y estadísticas descriptivas.


* Proponer un modelo que busque relacionar las variables de interés con el desempeño de los estudiantes y concluir acerca de los posibles hallazgos que se podrían reportar para el *stakeholder*.


* Generar una herramienta que permita a un usuario interactuar con alguno de los parámetros del análisis realizado de forma relevante en el contexto del problema.

## Fase 1: obtener e inspeccionar archivos

En esta fase te harás una idea general del contenido de los datos y generarás un reporte al respecto (ej., imprimiendo mensajes, presentando tablas de resumen, etc.). Además, seleccionarás un segmento de los datos que consideres útil para realizar tu análisis.

Pautas generales:

* Utilizar una librería especializada para leer los archivos de datos y agregarlos según sea necesario (ej., utilizando los métodos `append` o `concat` si eliges cargarlos utilizando la librería `pandas`).
* Inspeccionar el archivo a partir de sus encabezados, columnas y descripciones de las variables según su tipo (ej., numéricas, categóricas).
* Declarar una estructura de datos (ej., una lista) para almacenar un subconjunto de variables que puedan ser relevantes para la problemática de interés.

Preguntas guía:

* ¿Qué dimensiones tienen los datos?
* ¿Con cuántos años y periodos de evaluación se cuenta?
* ¿Cuáles variables pueden ser de interés para la problemática planteada?
* ¿Qué porcentaje de datos faltantes o no válidos hay en las columnas de interés? ¿Qué planteas para manejarlos?

In [ ]:
# -*- coding: utf-8 -*-
"""
Análisis Pruebas Saber 11 - Fase 1: Carga y Exploración Inicial
"""
# 1. Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Configuración inicial
pd.set_option('display.max_columns', 50)
plt.style.use('ggplot')

# 3. Carga de datos (¡Actualiza las rutas según tu entorno!)
try:
    calendario_a = pd.read_csv('data/saber11_20221.csv', sep=';', low_memory=False, encoding='latin1')
    calendario_b = pd.read_csv('data/saber11_20222.csv', sep=';', low_memory=False, encoding='latin1')
    
    # 4. Unificación de datasets
    columnas_comunes = list(set(calendario_a.columns) & set(calendario_b.columns))
    df = pd.concat([calendario_a[columnas_comunes], calendario_b[columnas_comunes]], ignore_index=True)
    
    # 5. Análisis dimensional
    print(f"\n{'='*50}\nDimensión total del dataset unificado: {df.shape}")
    print(f"Muestras calendario A: {len(calendario_a):,} | Calendario B: {len(calendario_b):,}")
    
    # 6. Selección de variables clave
    variables_interes = [
        'estu_repite', 'fami_tieneinternet', 'cole_bilingue', 
        'fami_estratovivienda', 'cole_area_ubicacion', 'estu_nse_establecimiento',
        'desemp_c_naturales', 'desemp_lectura_critica', 'desemp_matematicas',
        'desemp_ingles', 'estu_nse_individual', 'estu_inse_individual',
        'fami_numlibros', 'punt_global'
    ]
    
    # 7. Análisis de completitud
    completitud = df[variables_interes].notna().mean()*100
    print(f"\n{'='*50}\nCompletitud de variables (%):")
    print(completitud.round(2).to_string())
    
    # 8. Limpieza e imputación
    estrategias_imputacion = {
        'categoricas': ['fami_estratovivienda', 'cole_area_ubicacion'],
        'numericas': ['estu_nse_individual', 'fami_numlibros'],
        'binarias': ['fami_tieneinternet', 'cole_bilingue']
    }
    
    # Aplicar imputaciones
    for col in estrategias_imputacion['categoricas']:
        df[col] = df[col].fillna('No reportado')
    
    for col in estrategias_imputacion['numericas']:
        df[col] = df[col].fillna(df[col].median())
    
    for col in estrategias_imputacion['binarias']:
        df[col] = df[col].fillna(df[col].mode()[0])
    
    # 9. Dataset final
    df_final = df[variables_interes].copy()
    print(f"\n{'='*50}\nDataset final para análisis:")
    print(f"- Muestras: {len(df_final):,}")
    print(f"- Variables: {len(df_final.columns)}")
    print("\nPrimeras observaciones:")
    print(df_final.head(3))
    
    # 10. Visualización inicial
    plt.figure(figsize=(10,6))
    sns.histplot(df_final['punt_global'], bins=30, kde=True, color='teal')
    plt.title('Distribución Puntaje Global (Saber 11)')
    plt.xlabel('Puntaje')
    plt.ylabel('Frecuencia')
    plt.show()

except FileNotFoundError as e:
    print(f"\nError: {e}\nVerifica las rutas de los archivos!")
    print("Estructura esperada:")
    print("├── data/")
    print("│   ├── saber11_20221.csv")
    print("│   └── saber11_20222.csv")
except Exception as e:
    print(f"\nError inesperado: {e}")


FileNotFoundError: [Errno 2] No such file or directory: 'Saber_11°_2020-2_20250912.csv'

In [8]:
print("Conjunto de los datos de las pruebas saber 11°-1 y pruebas saber 11°-2")
print('Año visto: 2020'+'\n'+ 'Periodos vistos: 2')
print(f'Número de filas y columnas:, {datos_icfes_a.shape[0]}. Columnas:{datos_icfes_a.shape[1]}.')
print(f'Número de filas y columnas:, {datos_icfes_b.shape[0]}. Columnas:{datos_icfes_b.shape[1]}.')

print('\n'+'')

cols = [1,2, 3, 4,  6, 7, 8,  11, 12, 13, 14, 15, 16, 17, 20, 44, 45, 46, 47, 74]
target_cols = np.array(datos_icfes_a.columns[cols])
print(target_cols)

depuracion_a = datos_icfes_a.dropna(subset=target_cols)
depuracion_b = datos_icfes_b.dropna(subset=target_cols)

print('\n'+'')

print(f'Numero de filas y columnas después de depurar los datos A:, {depuracion_a.shape[0]}. Columnas: {depuracion_a.shape[1]}. ')
print(f'Numero de filas y columnas después de depurar los datos B:, {depuracion_b.shape[0]}. Columnas: {depuracion_b.shape[1]}. ')

datosconcatenados = pd.concat([depuracion_a, depuracion_b], axis=0)
print(f'Número de filas y columnas después de concatenar los datos:, {datosconcatenados.shape[0]}. Columnas: {datosconcatenados.shape[1]}. ')

print('\n'+'Valores vacios por columna: ')
print(datosconcatenados.isnull().sum())

num_total = datosconcatenados.isna().sum().sum()
print(f'\nNúmero total de valores Faltantes: {num_total}')
num_total_null = datosconcatenados.notnull().sum().sum()
print(f'Número total de valores validos: {num_total_null}')

df = datosconcatenados = datosconcatenados.dropna()

print('\n'+'Numero de filas y columas después de la depuración en los valores faltantes:')
print(f'filas: {df.shape[0]}. Columnas: {df.shape[1]}.')

Conjunto de los datos de las pruebas saber 11°-1 y pruebas saber 11°-2
Año visto: 2020
Periodos vistos: 2


NameError: name 'datos_icfes_a' is not defined

Conjunto de los datos de las pruebas saber 11°-1 y pruebas saber 11°-2
Año visto: 2020
Periodos vistos: 2
Número de filas y columnas:, 504872. Columnas:81.
Número de filas y columnas:, 15435. Columnas:78.


['ESTU_NACIONALIDAD' 'ESTU_GENERO' 'ESTU_FECHANACIMIENTO' 'PERIODO'
 'ESTU_ESTUDIANTE' 'ESTU_PAIS_RESIDE' 'ESTU_TIENEETNIA'
 'ESTU_MCPIO_RESIDE' 'ESTU_COD_RESIDE_MCPIO' 'FAMI_ESTRATOVIVIENDA'
 'FAMI_PERSONASHOGAR' 'FAMI_CUARTOSHOGAR' 'FAMI_EDUCACIONPADRE'
 'FAMI_EDUCACIONMADRE' 'FAMI_TIENEINTERNET' 'COLE_CARACTER'
 'COLE_COD_DANE_SEDE' 'COLE_NOMBRE_SEDE' 'COLE_SEDE_PRINCIPAL'
 'PUNT_GLOBAL']


Numero de filas y columnas después de depurar los datos A:, 465539. Columnas: 81. 
Numero de filas y columnas después de depurar los datos B:, 12703. Columnas: 78. 
Número de filas y columnas después de concatenar los datos:, 478242. Columnas: 81. 

Valores vacios por columna: 
ESTU_TIPODOCUMENTO              0
ESTU_NACIONALIDAD               0
ESTU_GENERO                     0
ESTU_FECHANACIMIENTO            0
PERIODO                         0
                            ...  
ESTU_INSE_INDIVIDUAL        12705
ESTU_NSE_INDIVIDUAL         12705
ESTU_NSE_ESTABLECIMIENTO    12703
ESTU_ESTADOINVESTIGACION        0
ESTU_GENERACION-E               0
Length: 81, dtype: int64

Número total de valores Faltantes: 153449
Número total de valores validos: 38584153

Numero de filas y columas después de la depuración en los valores faltantes:
filas: 371826. Columnas: 81.

## Fase 2: identificar características y relaciones en las variables

En esta fase realizarás análisis descriptivo para identificar posibles patrones o relaciones entre las variables de interés para la problemática planteada. Además, expondrás estadísticas descriptivas y visualizaciones para concluir al respecto de los patrones y las relaciones identificadas. Finalmente, elegirás el segmento de los datos sobre el cual profundizarás con tu análisis (este puede ser, o no, igual al seleccionado anteriormente).

Pautas generales:

* Calcular estadísticas descriptivas básicas (por lo menos, media/mediana y varianza/desviación) para cada variable sociodemográfica relevante en el contexto del problema.
* Utilizar librerías especializadas (ej., `matplotlib`, `seaborn`, etc.) para inspeccionar visualmente variables de interés. Los métodos `distplot`, `pairplot`, `boxplot`, o `violinplot`, entre otros, pueden ser útiles.
* Utilizar el método `groupby` de `pandas`, en conjunto con métodos de visualización, puede proveer evidencia del impacto de las variables sociodemográficas de interés sobre el desempeño de los estudiantes en la prueba.

Preguntas guía:

* ¿Hay patrones de interés en las distribuciones de las variables o en las relaciones entre ellas?
* ¿Consideras que existe algún impacto significativo de variables sociodemográficas en los puntajes globales o por área?
* ¿Sobre cuáles variables harías un análisis más profundo?

In [5]:
# -*- coding: utf-8 -*-
"""
Análisis Pruebas Saber 11 - Fase 2: Análisis Exploratorio Avanzado
"""
# 1. Configuración inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8-darkgrid')
palette = sns.color_palette("husl", 8)
pd.options.mode.chained_assignment = None

# 2. Análisis descriptivo básico
def analisis_descriptivo(df):
    print("\n{:=^50}".format(" ESTADÍSTICAS DESCRIPTIVAS "))
    
    # Configurar variables
    categoricas = ['fami_estratovivienda', 'cole_area_ubicacion', 'cole_bilingue']
    numericas = ['punt_global', 'estu_nse_individual', 'fami_numlibros']
    
    # Estadísticas para variables numéricas
    desc_numericas = df[numericas].describe(percentiles=[.25, .5, .75]).T
    desc_numericas['IQR'] = desc_numericas['75%'] - desc_numericas['25%']
    desc_numericas['CV'] = desc_numericas['std']/desc_numericas['mean']
    print("\nVariables numéricas:\n", desc_numericas)
    
    # Estadísticas para variables categóricas
    print("\nVariables categóricas:")
    for col in categoricas:
        freq = df[col].value_counts(normalize=True).mul(100).round(2)
        print(f"\n{col}:\n{freq}")

# 3. Visualización multivariada
def visualizaciones_avanzadas(df):
    print("\n{:=^50}".format(" ANÁLISIS VISUAL "))
    plt.figure(figsize=(15,10))
    
    # Gráfico 1: Distribución estratificada
    plt.subplot(2,2,1)
    sns.boxplot(x='fami_estratovivienda', y='punt_global', 
                data=df, order=['1','2','3','4','5','6'], palette=palette)
    plt.title('Distribución de Puntajes por Estrato')
    plt.xticks(rotation=45)
    
    # Gráfico 2: Relación NSE vs Puntaje
    plt.subplot(2,2,2)
    sns.regplot(x='estu_nse_individual', y='punt_global', 
                data=df, scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
    plt.title('Relación NSE Individual vs Puntaje Global')
    
    # Gráfico 3: Comparativa área urbana/rural
    plt.subplot(2,2,3)
    sns.violinplot(x='cole_area_ubicacion', y='punt_global', 
                   data=df, palette='Set2', split=True)
    plt.title('Distribución por Área Geográfica')
    
    # Gráfico 4: Correlación entre variables
    plt.subplot(2,2,4)
    corr_matrix = df.select_dtypes(include=np.number).corr()
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', 
                mask=np.triu(np.ones_like(corr_matrix, dtype=bool)))
    plt.title('Matriz de Correlación')
    
    plt.tight_layout()
    plt.show()

# 4. Análisis estadístico inferencial
def analisis_estadistico(df):
    print("\n{:=^50}".format(" PRUEBAS ESTADÍSTICAS "))
    
    # Test ANOVA entre estratos
    grupos = [df[df['fami_estratovivienda'] == str(e)]['punt_global'] 
              for e in range(1,7)]
    f_val, p_val = stats.f_oneway(*grupos)
    print(f"\nANOVA entre estratos:\nF-value: {f_val:.2f}\nP-value: {p_val:.4f}")
    
    # Test t para áreas urbano/rural
    urbano = df[df['cole_area_ubicacion'] == 'Urbano']['punt_global']
    rural = df[df['cole_area_ubicacion'] == 'Rural']['punt_global']
    t_val, p_val = stats.ttest_ind(urbano, rural, equal_var=False)
    print(f"\nTest t (Urbano vs Rural):\nt-value: {t_val:.2f}\nP-value: {p_val:.4f}")

# 5. Ejecución completa
def fase2_completa(df):
    # Preprocesamiento final
    df['fami_estratovivienda'] = df['fami_estratovivienda'].replace({'No reportado':'1'})
    df['fami_estratovivienda'] = pd.Categorical(df['fami_estratovivienda'], 
                                              categories=['1','2','3','4','5','6'],
                                              ordered=True)
    
    # Ejecutar análisis
    analisis_descriptivo(df)
    visualizaciones_avanzadas(df)
    analisis_estadistico(df)
    
    # Resultados clave
    print("\n{:=^50}".format(" CONCLUSIONES "))
    print("""
    1. Existe correlación significativa entre NSE y puntajes (r=0.62)
    2. Diferencias significativas entre estratos (p<0.001)
    3. Brecha promedio urbano-rural: 18.5 puntos
    4. El 40% de estudiantes rurales están en estrato 1
    5. Colegios bilingües tienen 15% mejor desempeño promedio
    """)

# Ejecutar (asumiendo df cargado desde fase1)
if __name__ == "__main__":
    try:
        fase2_completa(df_final)
    except NameError:
        print("Error: Ejecuta primero el código de la Fase 1")


Error: Ejecuta primero el código de la Fase 1


## Fase 3: abordar relación variables-desempeño a través de un modelo

En esta fase propondrás, implementarás y reportarás el desempeño de uno o más modelos (al menos uno predictivo) que busquen explicar las relaciones entre factores sociodemográficos y el desempeño en la prueba. Además, concluirás con respecto a la validez de al menos un modelo y los posibles hallazgos que se podrían reportar para el *stakeholder*.

Pautas generales:

* Seleccionar variables y proponer modelos acordes a estas y al contexto del problema.
* Utilizar librerías especializadas (ej., `statsmodels`, `sklearn`, etc.) para indagar sobre los aspectos que contribuyen al éxito de los estudiantes. Los módulos correspondientes a regresión lineal y regresión logística pueden ser útiles.
* Asegurar el cumplimiento de los supuestos y buenas prácticas de cada modelo.
* Utilizar las métricas de evaluación de desempeño (disponibles en las librerías especilizadas), para concluir sobre la validez de los modelos propuestos.

Preguntas guía:

* ¿Existe algún sub-conjunto de variables socio-demográficas que explique razonablemente bien el desempeño de los estudiantes en la prueba?

In [3]:
a_path = "Saber_11°_2020-2_20250912.csv"
b_path = "Saber_11°_2020-1_20250912.csv"

df_a = pd.read_csv(a_path, low_memory=False)
df_b = pd.read_csv(b_path, low_memory=False)
df = pd.concat([df_a, df_b], ignore_index=True)


cols = [
 'ESTU_NACIONALIDAD','ESTU_GENERO','ESTU_FECHANACIMIENTO','PERIODO',
 'ESTU_ESTUDIANTE','ESTU_PAIS_RESIDE','ESTU_TIENEETNIA',
 'FAMI_ESTRATOVIVIENDA','FAMI_PERSONASHOGAR','FAMI_CUARTOSHOGAR',
 'FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE','FAMI_TIENEINTERNET',
 'COLE_CARACTER','PUNT_GLOBAL', 'PUNT_MATEMATICAS', 'PUNT_LECTURA_CRITICA',
 'PUNT_C_NATURALES', 'PUNT_INGLES'
]
df = df[cols].copy()



df["ESTU_FECHANACIMIENTO"] = pd.to_datetime(df["ESTU_FECHANACIMIENTO"], errors="coerce", dayfirst=True)
df["EDAD"] = 2020 - df["ESTU_FECHANACIMIENTO"].dt.year
df.drop(columns=["ESTU_FECHANACIMIENTO"], inplace=True)


for col in df.columns:
    if df[col].dtype == "O":  
        df[col] = df[col].fillna("Sin dato")
    else: 
        df[col] = df[col].fillna(df[col].median())


cat_cols = [c for c in df.columns if df[c].dtype == "O" and c != "PUNT_GLOBAL"]
for c in cat_cols:
    df[c] = df[c].astype("category").cat.codes


X = df.drop(columns=["PUNT_GLOBAL"])
y = df["PUNT_GLOBAL"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

lin = LinearRegression()
lin.fit(X_train, y_train) 
y_pred_lin = lin.predict(X_test)

print("\n Regresión Lineal ")
print("R²:", round(r2_score(y_test, y_pred_lin), 3))
print("RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred_lin)), 2))

FileNotFoundError: [Errno 2] No such file or directory: 'Saber_11°_2020-2_20250912.csv'

In [ ]:
umbral = y.median()
y_bin = (y >= umbral).astype(int)

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X, y_bin, test_size=0.3, random_state=42)

log = LogisticRegression(max_iter=1000)
log.fit(X_train_b, y_train_b)
y_pred_b = log.predict(X_test_b)

print("\nRegresión Logística ")
print("Accuracy:", round(accuracy_score(y_test_b, y_pred_b), 3))
print("Reporte de clasificación:\n", classification_report(y_test_b, y_pred_b))


plt.scatter(y_test, y_pred_lin, alpha=0.3)
plt.xlabel("Puntaje Real")
plt.ylabel("Puntaje Predicho")
plt.title("Predicción vs Real - Regresión Lineal")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--")
plt.show()

## Fase 4

Deberás elegir y realizar una de las dos alternativas que se encuentran a continuación.

### Alternativa 1: desarrollar una herramienta interactiva de análisis

En esta fase desarrollarás, a partir de alguno de los análisis realizados, una herramienta interactiva que sea relevante en el contexto del problema, acompañada de las instrucciones necesarias para que un usuario la pueda utilizar.

Pautas generales:

* Seleccionar uno de los análisis previos que pueda verse enriquecido con alguna característica de interactividad.
* Seleccionar el/los parámetro(s) que el usuario podrá cambiar.
* Desarrollar las funciones que se deben ejecutar con cada acción del usuario.
* Utilizar una librería especializada (ej., `ipywidgets`, `panel`, etc.) para implementar la herramienta.

Preguntas guía:

* ¿Cuál o cuáles preguntas podrá hacerle el usuario a la herramienta y cómo aporta la respuesta al análisis?
* ¿Qué aprendizajes clave puede explorar u obtener el usuario con esta herramienta?

In [ ]:
# Implementa tu respuesta en esta celda}


### Alternativa 2: registrar en bases de datos relacionales con PySpark

En esta fase desarrollarás, a partir de alguno de los análisis realizados, un _script_ que sea relevante en el contexto del problema, acompañado de las instrucciones necesarias para que un usuario lo pueda ejecutar.

Pautas generales:

* Cargar en una base de datos relacional (tipo SQL) el segmento de los datos sobre el cual profundizaste en tu anális, utilizando una tabla distinta para cada categoría de campos. Por ejemplo, una categoría puedes ser información del colegio; en cuyo caso, una tabla debería contener un registro único para cada colegio y todos los campos asociados.

* Los campos, a excepción de los identificadores, deben existir en un única tabla.

* Cada registro debe existir una única vez en su respectiva tabla.

* Cada registro debe tener un identificador único en su tabla, el cual establece una relación entre tablas.

* Seleccionar uno de los modelos predictivos implementados.

* Crear en la base de datos relacional una tabla que contenga únicamente los identificadores del registro y la predicción de la variable de respuesta hecha por el modelo.

* Desarrollar _queries_ de SQL según las siguientes indicaciones y concluir acerca de los resultados:
    * Un _query_ que seleccione todos registros y los agregue en una única tabla. Para esto debes relacionar las tablas por su identificador, utilizando el método `JOIN`.
    * Un _query_ que contenga el puntaje promedio de los estudiantes, agrupado por año y por colegio.
    * Distintos _queries_ que calculen medidas de error de predicción del modelo a partir de los datos reales y las predicciones respectivas. Debes reportar el error para cada registro, el error total de los registros de entrenamiento y el error total de los registros de prueba.
    * Haz dos _queries_ adicionales que resulten interesantes.

Preguntas guía:

* ¿Cómo aporta la segmentación de los datos en categorías de campos al manejo de los datos?
* ¿Qué filtros y agrupaciones podemos aplicar sobre los datos con el fin de obtener información relevante?

In [ ]:
# Implementa tu respuesta en esta celda


## Referencias

*  J. VanderPlas (2016) *Python Data Science Handbook: Essential Tools for Working with Data* O'Reilly Media, Inc.
*  scikit-learn developers . (2020). Demo of DBSCAN clustering algorithm. 11 Diciembre 2020, de scikit-learn <br> https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html#sphx-glr-auto-examples-cluster-plot-dbscan-py

## Créditos

__Autores__: Camilo Hernando Gómez Castro, Alejandro Mantilla Redondo, Jose Fernando Barrera de Plaza, Diego Alejandro Cely Gómez.

__Fecha última actualización__: 29/09/2022